In [ ]:
import gc

import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

* agg 데이터 만들기
    * kaggle datasets 에서 가져와서 사용 가능.

In [ ]:
# train = pd.read_parquet('./data/train.parquet')
# train_labels = pd.read_csv('./data/train_labels.csv')

# train.shape, train_labels.shape

In [ ]:
# train = train.merge(train_labels, how='inner', on='customer_ID')
# print(train.shape)
# del train_labels
# gc.collect()

In [ ]:
# features = train.drop(['customer_ID', 'S_2', 'target'], axis=1).columns.to_list()
# categorical_features = [
#     'B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68',
#     'D_114', 'D_116', 'D_117', 'D_120', 'D_126'
# ]
# num_features = [col for col in features if col not in categorical_features]

In [ ]:
# train_num_agg = train.groupby('customer_ID')[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
# train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
# train_cat_agg = train.groupby('customer_ID')[categorical_features].agg(['count', 'last', 'nunique'])
# train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
# train_target = (train.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()['target'])
# train = pd.concat([train_num_agg, train_cat_agg, train_target], axis=1)

# train.to_parquet('./data/train_agg.parquet', engine='pyarrow', compression='gzip', index=False)
# del train_num_agg, train_cat_agg, train_target
# gc.collect()

In [ ]:
# test = pd.read_parquet('./data/test.parquet')
# test.shape

In [ ]:
# test_num_agg = test.groupby('customer_ID')[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
# test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
# test_cat_agg = test.groupby('customer_ID')[categorical_features].agg(['count', 'last', 'nunique'])
# test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
# test = pd.concat([test_num_agg, test_cat_agg], axis=1)

# test.to_parquet('./data/test_agg.parquet', engine='pyarrow', compression='gzip', index=False)
# del test_num_agg, test_cat_agg
# gc.collect()

In [ ]:
# train.head()

* train, test agg 데이터 불러오기

In [ ]:
train = pd.read_parquet('./data/train_agg.parquet')
test = pd.read_parquet('./data/test_agg.parquet')

train.shape, test.shape

* 평가지표
    * https://www.kaggle.com/competitions/amex-default-prediction/overview/evaluation

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
            .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x == 0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()

    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
            .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x == 0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

##### 범주형 특성 LabelEncoding

In [ ]:
features = test.columns.to_list()
categorical_features = [
    'B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68',
    'D_114', 'D_116', 'D_117', 'D_120', 'D_126'
]
categorical_features = [f'{cf}_last' for cf in categorical_features]
le = LabelEncoder()
for categorical_feature in categorical_features:
    train[categorical_feature] = le.fit_transform(train[categorical_feature])
    test[categorical_feature] = le.transform(test[categorical_feature])

In [ ]:
y_train = pd.DataFrame(train['target'])
X_train = train.drop('target', axis=1)

##### catboost 학습
    * StratifiedKFold 이용해서 5개의 모델 학습
    * CatBoostClassifier
        * task_type='GPU' 이용해서 GPU 통해 학습 가능.
    * deep copy
        * 메모리 내용에 있는 값을 복사.

In [ ]:
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=22)
y_oof = np.zeros(X_train.shape[0])
y_test = np.zeros(test.shape[0])
idx = 0

for train_idx, val_idx in skf.split(X_train, y_train):
    print(f'******* Fold {idx} *******')
    X_tr, X_val = (
        X_train.iloc[train_idx].reset_index(drop=True),
        X_train.iloc[val_idx].reset_index(drop=True)
    )
    y_tr, y_val = (
        y_train.iloc[train_idx].reset_index(drop=True),
        y_train.iloc[val_idx].reset_index(drop=True)
    )

    clf = CatBoostClassifier(iterations=5000, random_state=22, task_type='GPU')
    clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], cat_features=categorical_features, verbose=100)
    preds = clf.predict_proba(X_val)[:, 1]
    y_oof[val_idx] = y_oof[val_idx] + preds

    preds_test = clf.predict_proba(test)[:, 1]
    y_test = y_test + preds_test / N_FOLDS
    idx += 1

y_pred = y_train.copy(deep=True)
y_pred = y_pred.rename(columns={'target': 'prediction'})
y_pred['prediction'] = y_oof
val_score = amex_metric(y_train, y_pred)
print(f'Amex metric: {val_score}')

In [ ]:
y_oof_binary = (y_oof >= np.percentile(y_oof, 96)).astype(int)

In [ ]:
y_oof_binary.mean()

In [ ]:
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment='center', color='white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cm = confusion_matrix(y_train, y_oof_binary)
class_names = [0, 1]
plt.figure()
plot_confusion_matrix(cm, classes=class_names, title=f'Confusion matrix at 4%')
plt.show()

In [ ]:
test['prediction'] = y_test
test['prediction'].to_csv(f'submission_cat_{val_score}.csv', index=True)